In [5]:
# %pip install -e ../..
# %pip show lightgbm

In [6]:
from data.load import load_data
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split

In [7]:

# Load data
# svo = [('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg'), ('prop_id', 'price_usd', 'avg'), ('srch_destination_id', 'price_usd', 'avg'), ('site_id', 'prop_starrating', 'avg'), ('srch_id', 'prop_starrating', 'avg'), ('prop_id', 'prop_starrating', 'avg'), ('srch_destination_id', 'prop_starrating', 'avg'), ('site_id', 'prop_review_score', 'avg'), ('srch_id', 'prop_review_score', 'avg'), ('prop_id', 'prop_review_score', 'avg'), ('srch_destination_id', 'prop_review_score', 'avg'), ('site_id', 'prop_log_historical_price', 'avg'), ('srch_id', 'prop_log_historical_price', 'avg'), ('prop_id', 'prop_log_historical_price', 'avg'), ('srch_destination_id', 'prop_log_historical_price', 'avg'),
# ('srch_destination_id', 'prop_location_score1', 'avg'),('prop_id', 'prop_location_score1', 'avg'),('srch_id', 'prop_location_score1', 'avg'),('site_id', 'prop_location_score1', 'avg')]

# fts=[('prop_starrating', 'visitor_hist_starrating', 'diff'), ('same_val_prop_id_price_usd_avg', 'same_val_prop_id_prop_log_historical_price_avg', 'all'), ('same_val_prop_id_price_usd_avg', 'prop_review_score', 'all'), ('same_val_srch_destination_id_price_usd_avg', 'same_val_srch_destination_id_prop_log_historical_price_avg', 'all'), ('same_val_srch_destination_id_price_usd_avg', 'prop_review_score', 'all'), ('prop_location_score2', 'prop_location_score1', 'all')]

# order = [('srch_id', 'all'), ('prop_id', 'price_usd')]

# # This gives 0.524 at 50's!

# df = load_data(mode='kaggle_test', add_day_parts=True, same_value_operations=svo, fts_operations=fts, add_seasons=True, add_dist_holidays=True, scaling=True, order_operations=order, caching=True)
# df.to_csv('test_ext_fts.csv')

In [8]:
df = pd.read_csv('ext_fts.csv')

In [9]:
# Split into target and predictors
y = df['booking_bool']*10 + df['click_bool']
X = df.drop(['booking_bool','click_bool', 'gross_bookings_usd', 'date_time'], axis=1)
X = X.fillna(X.mean())

In [10]:
del df

In [11]:
import re
X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [12]:
X_train = X[:4900000]
X_eval = X[4900000:]
y_train = y[:4900000]
y_eval = y[4900000:]

qids_train = X_train.groupby('srch_id')['srch_id'].count().to_numpy()
qids_eval = X_eval.groupby('srch_id')['srch_id'].count().to_numpy()


In [13]:
from datetime import datetime
import lightgbm
model = lightgbm.LGBMRanker(verbose=3, objective='lambdarank', n_estimators=600,  n_jobs=8)
model.fit(
X=X_train,
y=y_train,
group=qids_train,
eval_set=[(X_eval, y_eval)],
eval_group=[qids_eval],
eval_at=[5, 10, 50],
early_stopping_rounds=100,
verbose= True)

/Users/sethvanderbijl/Coding Projects/DM_project_2/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/sethvanderbijl/Coding Projects/DM_project_2/.venv/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.914306
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.160822
[LightGBM] [Debug] init for col-wise cost 0.245150 seconds, init for row-wise cost 2.344663 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.731620 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 15721
[LightGBM] [Info] Number of data points in the train set: 4900000, number of used features: 181
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[1]	valid_0's ndcg@5: 0.356728	valid_0's ndcg@10: 0.424533	valid_0's ndcg@50: 0.487028
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[2]	valid_0's ndcg@5: 0.380698	valid_0's ndcg@10: 0.443797	valid_0's ndcg@50: 0.504225
[LightGBM] [Debug] Trained a tree 

LGBMRanker(n_estimators=600, n_jobs=8, objective='lambdarank', verbose=3)

In [14]:
model.best_score_['valid_0']['ndcg@5']

0.4412215301387838

In [15]:
# Feature importances
t=zip(model.feature_name_, model.feature_importances_)
t = pd.DataFrame(t, columns=['n','i']).sort_values('i', ascending= False)
print(t.to_numpy())

[['order_srch_id_price_usd' 437]
 ['price_usd' 419]
 ['position' 243]
 ['prop_location_score2' 216]
 ['order_prop_id_price_usd' 176]
 ['prop_log_historical_price' 166]
 ['order_srch_id_prop_location_score2' 153]
 ['prop_starrating' 141]
 ['prop_location_score2_diff_prop_location_score1' 132]
 ['random_bool' 127]
 ['prop_location_score2_div_prop_location_score1' 120]
 ['same_val_prop_id_price_usd_avg_mult_prop_review_score' 117]
 ['visitor_hist_adr_usd' 117]
 ['order_srch_id_same_val_prop_id_price_usd_avg' 116]
 ['prop_review_score' 107]
 ['srch_query_affinity_score' 106]
 ['order_srch_id_prop_starrating' 93]
 ['order_srch_id_prop_log_historical_price' 89]
 ['same_val_srch_id_price_usd_avg' 88]
 ['same_val_prop_id_price_usd_avg_div_prop_review_score' 87]
 ['order_srch_id_srch_query_affinity_score' 82]
 ['same_val_prop_id_price_usd_avg_div_same_val_prop_id_prop_log_historical_price_avg'
  76]
 ['srch_booking_window' 75]
 ['orig_destination_distance' 69]
 ['same_val_srch_id_prop_location_

In [16]:
X_train

,Unnamed0,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,same_val_prop_id_price_usd_avg_mult_prop_review_score,same_val_srch_destination_id_price_usd_avg_diff_same_val_srch_destination_id_prop_log_historical_price_avg,same_val_srch_destination_id_price_usd_avg_div_same_val_srch_destination_id_prop_log_historical_price_avg,same_val_srch_destination_id_price_usd_avg_mult_same_val_srch_destination_id_prop_log_historical_price_avg,same_val_srch_destination_id_price_usd_avg_diff_prop_review_score,same_val_srch_destination_id_price_usd_avg_div_prop_review_score,same_val_srch_destination_id_price_usd_avg_mult_prop_review_score,prop_location_score2_diff_prop_location_score1,prop_location_score2_div_prop_location_score1,prop_location_score2_mult_prop_location_score1
0,2956875,1,0.333333,0.808696,0.547120,0.089845,0.951965,59526,0.6,0.7,...,0.000065,0.000071,0.000216,0.000205,0.000072,0.000042,0.000057,0.313946,0.011824,0.012401
1,2956870,1,0.333333,0.808696,0.547120,0.089845,0.951965,27348,0.4,0.8,...,0.000056,0.000071,0.000216,0.000205,0.000071,0.000036,0.000066,0.409074,0.003048,0.005289
2,2956871,1,0.333333,0.808696,0.547120,0.089845,0.951965,29604,0.8,0.7,...,0.000082,0.000071,0.000216,0.000205,0.000072,0.000042,0.000057,0.365285,0.032435,0.048986
3,2956872,1,0.333333,0.808696,0.547120,0.089845,0.951965,50984,0.4,0.0,...,0.000000,0.000071,0.000216,0.000205,0.000074,0.000072,0.000000,0.424404,0.028961,0.075196
4,2956895,1,0.333333,0.808696,0.547120,0.089845,0.951965,893,0.6,0.7,...,0.000071,0.000071,0.000216,0.000205,0.000072,0.000042,0.000057,0.404530,0.010679,0.018533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4899995,1654024,329060,0.424242,0.234783,0.635097,0.086787,0.235808,113900,0.4,0.7,...,0.000040,0.000059,0.000195,0.000162,0.000060,0.000035,0.000049,0.401495,0.001171,0.001947
4899996,1654036,329060,0.424242,0.234783,0.635097,0.086787,0.235808,14688,0.6,0.6,...,0.000032,0.000059,0.000195,0.000162,0.000060,0.000041,0.000042,0.500370,0.004713,0.012296
4899997,1654037,329060,0.424242,0.234783,0.635097,0.086787,0.235808,14595,0.6,0.8,...,0.000063,0.000059,0.000195,0.000162,0.000060,0.000031,0.000055,0.415303,0.007067,0.012790
4899998,1654049,329060,0.424242,0.234783,0.635097,0.086787,0.235808,69550,0.8,0.7,...,0.000075,0.000059,0.000195,0.000162,0.000060,0.000035,0.000049,0.551623,0.019535,0.064717


KNeighborsRegressor()

In [18]:
# # Load data
# svo = [('site_id', 'price_usd', 'avg'), ('srch_id', 'price_usd', 'avg'), ('prop_id', 'price_usd', 'avg'), ('srch_destination_id', 'price_usd', 'avg'), ('site_id', 'prop_starrating', 'avg'), ('srch_id', 'prop_starrating', 'avg'), ('prop_id', 'prop_starrating', 'avg'), ('srch_destination_id', 'prop_starrating', 'avg'), ('site_id', 'prop_review_score', 'avg'), ('srch_id', 'prop_review_score', 'avg'), ('prop_id', 'prop_review_score', 'avg'), ('srch_destination_id', 'prop_review_score', 'avg'), ('site_id', 'prop_log_historical_price', 'avg'), ('srch_id', 'prop_log_historical_price', 'avg'), ('prop_id', 'prop_log_historical_price', 'avg'), ('srch_destination_id', 'prop_log_historical_price', 'avg'),
# ('srch_destination_id', 'prop_location_score1', 'avg'),('prop_id', 'prop_location_score1', 'avg'),('srch_id', 'prop_location_score1', 'avg'),('site_id', 'prop_location_score1', 'avg')]

# fts=[('prop_starrating', 'visitor_hist_starrating', 'diff'), ('same_val_prop_id_price_usd_avg', 'same_val_prop_id_prop_log_historical_price_avg', 'all'), ('same_val_prop_id_price_usd_avg', 'prop_review_score', 'all'), ('same_val_srch_destination_id_price_usd_avg', 'same_val_srch_destination_id_prop_log_historical_price_avg', 'all'), ('same_val_srch_destination_id_price_usd_avg', 'prop_review_score', 'all'), ('prop_location_score2', 'prop_location_score1', 'all')]

# order = [('srch_id', 'all'), ('prop_id', 'price_usd')]

# test = load_data(mode='kaggle_test', add_day_parts=True, same_value_operations=svo, fts_operations=fts, add_seasons=True, add_dist_holidays=True, scaling=True, order_operations=order, caching=True)

# test['position'] = position_estimator.predict(test.drop('date_time', axis=1))

# test.to_csv('ft_eng_test.csv')
# # df = pd.read_csv('for_colab.csv')

In [2]:
import pandas as pd

In [17]:
from sklearn.neighbors import KNeighborsRegressor
position_estimator = KNeighborsRegressor(n_neighbors=1)
X = pd.read_csv('/Users/sethvanderbijl/Coding Projects/DM_project_2/models/LambdaSeth/ext_fts.csv').drop(['date_time', 'booking_bool', 'click_bool', 'gross_bookings_usd'], axis=1)
# 'booking_bool','click_bool', 'gross_bookings_usd', 'date_time' are already dropped


In [18]:
X=X.fillna(X.mean())

In [19]:

position_estimator.fit(X.drop(['position'], axis=1), X['position'])

KNeighborsRegressor(n_neighbors=1)

In [20]:
test = pd.read_csv('/Users/sethvanderbijl/Coding Projects/DM_project_2/models/LambdaSeth/test_ext_fts.csv')

In [21]:
test = test.drop(['date_time'], axis=1)

In [22]:

test = test.fillna(X.mean())
del X

In [23]:
import tqdm.notebook as tq
import numpy as np
batch_size = 1000
result = []
for batch_number, batch_df in tq.tqdm(test.groupby(np.arange(len(test)) // batch_size)):
    result.append(position_estimator.predict(batch_df))

  0%|          | 0/4960 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [38]:
test['position'] = result

KeyboardInterrupt: 

In [ ]:
# results = model.predict(X_test)
# print(len(results.tolist()))

# new_df['predictions'] = results

In [ ]:
test.to_csv('with_pos.csv')

In [20]:
# # Drop irrelevant columns from our own test data
# test = test.drop(['date_time'], axis=1)
# X_test = test.fillna(X.mean()) #Mean of x or mean of x_test?
# del test

# results = model.predict(X_test)
# print(len(results.tolist()))

In [21]:
# # Check if there is a difference in columns (there shouldn't be)
# list(set(X_test.columns) - set(X.columns))

In [22]:


# new_df = X_test[['prop_id', 'srch_id']]
# new_df['predictions'] = results
# print(new_df.head(30))


In [23]:
# sorted_df = new_df.sort_values(['srch_id', 'predictions'], ascending=[True, False])
# print(sorted_df.head(30))

In [24]:
# export_df = sorted_df.drop(['predictions'], axis=1)
# export_df = export_df[['srch_id', 'prop_id']] # swap columns for required format

# compression_opts = dict(method='zip', archive_name='out.csv')  
# export_df.to_csv('out.zip', index=False, compression=compression_opts) # we can submit .zip instead of the raw .csv, which saves some space.